In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cat /proc/meminfo

MemTotal:       37066884 kB
MemFree:        33655452 kB
MemAvailable:   35500700 kB
Buffers:           79648 kB
Cached:          2041056 kB
SwapCached:            0 kB
Active:           807616 kB
Inactive:        1787712 kB
Active(anon):     442740 kB
Inactive(anon):      344 kB
Active(file):     364876 kB
Inactive(file):  1787368 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               284 kB
Writeback:             0 kB
AnonPages:        474792 kB
Mapped:           247988 kB
Shmem:               904 kB
Slab:             400876 kB
SReclaimable:     174316 kB
SUnreclaim:       226560 kB
KernelStack:       14280 kB
PageTables:         8560 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    18533440 kB
Committed_AS:    8118364 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:            18400 kB
AnonHugePages:   

In [ ]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.10.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 40205 bytes (39 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

In [ ]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [ ]:
%%R
path  <-  '/content/drive/My Drive/mhar_paper_samples_2/'
set.seed(5)
simplex_gen(5,10)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [ ]:
%%R

file_name <- files[1]
perm <- 50
n_sample <-5000
print(file_name)
df_  <- read_csv(file_name)
df_ <- df_[c(1:n_sample),]
# Get dim of the read file
n_rows  <- dim(df_)[1]
n_cols  <- dim(df_)[2]
print(n_rows)
print(n_cols)

if (grepl('hyper', files[1], fixed = TRUE)){
  real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
}
if (grepl('simplex', files[1], fixed = TRUE)){
  real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
}

df_  <- df_  %>% data.matrix()   %>% unname()
        
df_  <- rbind(df_, real_sample)  
df_  <- df_ %>% aperm(c(2,1)) 
dim(df_)
result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=2, pvalue.only=FALSE)
z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
gc()

In [ ]:
%%R
result